In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [21]:
model = AutoModelForSequenceClassification.from_pretrained("yjoonjang/preranker", num_labels=2).to("cuda:1")
tokenizer = AutoTokenizer.from_pretrained("yjoonjang/preranker")

Some weights of the model checkpoint at yjoonjang/preranker were not used when initializing XLMRobertaForSequenceClassification: ['bio_classifier.0.bias', 'bio_classifier.0.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
query = "what is panda?"
passage = "The giant panda, sometimes called a panda bear or simply panda, is a bear species endemic to China."

pair = [(query, passage)]

In [23]:
feature = tokenizer(
	pair,
	padding=True,
	truncation=True,
	max_length=8194,
	return_tensors="pt"
)
feature.items()

dict_items([('input_ids', tensor([[     0,   2367,     83,      6,  85407,     32,      2,      2,    581,
           6051,     18,      6,  85407,      4,  68018,  35839,     10,      6,
          85407,  81148,    707,  42856,      6,  85407,      4,     83,     10,
          81148, 114149,  28117,  21068,     47,   9098,      5,      2]])), ('attention_mask', tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))])

In [24]:
feature = {k: v.to("cuda:1") for k, v in feature.items()}

In [25]:
outputs = model(**feature, return_dict=True)
outputs


SequenceClassifierOutput(loss=None, logits=tensor([[-5.3474,  5.3474]], device='cuda:1', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [27]:
import torch

logits = outputs.logits
probs = torch.softmax(logits, dim=1)

probs

tensor([[2.2660e-05, 9.9998e-01]], device='cuda:1', grad_fn=<SoftmaxBackward0>)

In [29]:
class_ids = torch.argmax(logits, dim=1)

class_ids

tensor([1], device='cuda:1')

: 